In [13]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

## Models and layers extend `nn.Module` and implement `__init__` and `forward`

In [2]:
class SimpleClassifier(nn.Module):
    def __init__(self, in_dim, h_dim, out_dim):
        super(SimpleClassifier, self).__init__()
        self.l1 = nn.Linear(in_dim, h_dim)
        self.l2 = nn.Linear(h_dim, out_dim)
    
    def forward(self, x):
        x = self.l1(x)
        x = F.sigmoid(x)
        
        x = F.softmax(self.l2(x))
        return x

In [3]:
# Constants for cleanliness
n_classes = 5
n_features = 3
n_examples = 5
hidden_size = 4

## Create an instance of our model

In [4]:
model = SimpleClassifier(n_features, hidden_size, n_classes)

## Create some random data. Data fed to models must be `Variable`s

In [5]:
x_train = Variable(torch.rand(n_examples, n_features))
# Create a tensor of n_examples length with random integers of value up to the n_classes
# this is just what is expected by loss functions that work with softmax
y_train = Variable(torch.LongTensor(n_examples).random_(n_classes))

In [6]:
print(x_train, y_train)

Variable containing:
 0.2265  0.7369  0.5146
 0.1940  0.1915  0.2595
 0.2240  0.8522  0.2573
 0.2369  0.1679  0.9709
 0.6123  0.5646  0.4865
[torch.FloatTensor of size 5x3]
 Variable containing:
 4
 0
 1
 2
 4
[torch.LongTensor of size 5]



## Evaluate our model

In [7]:
y_hat = model(x_train)

## Check our loss

In [8]:
criterion = nn.CrossEntropyLoss()

In [9]:
loss = criterion(y_hat, y_train)
print('loss:', loss.data[0])

loss: 1.6610389947891235


## Lets train it a bit, see if we can learn random

In [10]:
criterion = nn.CrossEntropyLoss()
model = SimpleClassifier(n_features, hidden_size, n_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [11]:
# generator that produces random data
def data_batch_generator(n_epoch, n_examples, n_features, n_classes):
    for i in range(n_epoch):
        x = Variable(torch.rand(n_examples, n_features))
        y = Variable(torch.LongTensor(n_examples).random_(n_classes))
        yield (x, y)

In [12]:
n_epochs = 10
n_classes = 5
n_features = 3
n_examples = 5
hidden_size = 4

gen = data_batch_generator(n_epochs, n_examples, n_features, n_classes)

for i, (x, y) in enumerate(gen):
    # eval model, calculate loss
    y_hat = model(x)
    loss = criterion(y_hat, y)
    print('epoch: {}, loss: {}'.format(i, loss.data[0]))
    
    # backprop error, update weights, zero old grads
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

epoch: 0, loss: 1.5883047580718994
epoch: 1, loss: 1.613193154335022
epoch: 2, loss: 1.6308625936508179
epoch: 3, loss: 1.7212693691253662
epoch: 4, loss: 1.5858738422393799
epoch: 5, loss: 1.6628410816192627
epoch: 6, loss: 1.5932284593582153
epoch: 7, loss: 1.6379722356796265
epoch: 8, loss: 1.546557068824768
epoch: 9, loss: 1.6378053426742554
